How you can use EvalML to automatically find the best pipeline for predicting whether or not a credit card transaction is fradulent.

In [9]:
#import dependencies 
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types
import woodwork as ww



In [10]:
#pulling in demo data.
X, y = evalml.demos.load_fraud(n_rows=250)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 250
Targets
False    88.40%
True     11.60%
Name: fraud, dtype: object


In [11]:
#To check the X or y add .ww for the woodwork multidementional data typing
X.ww
#Or 
y.ww

<Series: fraud (Physical Type = bool) (Logical Type = Boolean) (Semantic Tags = set())>

In [12]:
#Now to reformat the table 
X.ww["expiration_date"] = X["expiration_date"].apply(
    lambda x: "20{}-01-{}".format(x.split("/")[1], x.split("/")[0])
)
X = infer_feature_types(X,
                        feature_types={
                            "store_id":"categorical",
                            "expiration_date":"datetime",
                            "lat":"categorical",
                            "long":"categorical",
                            "provider":"categorical",
                        },
                        )
X.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
card_id,int64,Integer,['numeric']
store_id,int64,Integer,['numeric']
datetime,datetime64[ns],Datetime,[]
amount,int64,Integer,['numeric']
currency,string,Unknown,[]
customer_present,bool,Boolean,[]
expiration_date,datetime64[ns],Datetime,[]
provider,category,Categorical,['category']
lat,float64,Double,['numeric']


In [14]:
# To validate the result of the pipeline creation save some of the data as a holdout set
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(
    X,y, problem_type="binary", test_size=0.2
)